In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
import gc
import random
import os
import warnings
warnings.filterwarnings('ignore')

# ============================================
# 📌 0. Seed 고정 (재현성 보장)
# ============================================
SEED = 42

# Python hash seed
os.environ['PYTHONHASHSEED'] = str(SEED)

# Python random
random.seed(SEED)

# NumPy
np.random.seed(SEED)

# PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 추가적인 재현성 보장
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # CUDA 재현성

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================
# 유틸리티 함수
# ============================================
def clean_numeric(df):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str).str.replace(",", "")
            try:
                df[col] = pd.to_numeric(df[col])
            except (ValueError, TypeError):
                pass
    return df

class EarlyStopping:
    def __init__(self, patience=15, min_delta=0.0001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_state = model.state_dict().copy()
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def load_best_model(self, model):
        if self.best_model_state is not None:
            model.load_state_dict(self.best_model_state)

# ============================================
# 데이터 로드
# ============================================
TRAIN_PATH = "/content/drive/MyDrive/Colab Notebooks/POS_train_val.csv"
TEST_PATH = "/content/drive/MyDrive/Colab Notebooks/POS_test.csv"
ACAD_PATH = "/content/drive/MyDrive/Colab Notebooks/학사일정_정리(2325).csv"

print("=" * 60)
print("데이터 로드 중...")
print("=" * 60)

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
acad = pd.read_csv(ACAD_PATH)

train["영업일자"] = pd.to_datetime(train["영업일자"])
test["영업일자"] = pd.to_datetime(test["영업일자"])
acad["영업일자"] = pd.to_datetime(acad["date"])
acad_cols = [c for c in acad.columns if c != "영업일자" and c != "date"]
acad = acad.drop(columns=["date"])

train = train.merge(acad, on="영업일자", how="left")
test = test.merge(acad, on="영업일자", how="left")

train = train.sort_values("영업일자").reset_index(drop=True)
test = test.sort_values("영업일자").reset_index(drop=True)

print(f"\n학사일정 데이터 컬럼: {acad_cols}")

# 일매출 정리
train["일매출"] = train["일매출"].astype(str).str.replace(",", "").str.replace(" ", "")
train = clean_numeric(train)
train["일매출"] = pd.to_numeric(train["일매출"], errors='coerce').fillna(0)

if "일매출" in test.columns:
    test["일매출"] = test["일매출"].astype(str).str.replace(",", "").str.replace(" ", "")
    test = clean_numeric(test)
    test["일매출"] = pd.to_numeric(test["일매출"], errors='coerce').fillna(0)

# ============================================================================
# ============================================================================
# PART 1: 첫 번째 코드 - LSTM (그대로 이어붙이기)
# ============================================================================
# ============================================================================

print("\n" + "=" * 60)
print("PART 1: LSTM 모델 학습 및 예측 (첫 번째 코드 그대로)")
print("=" * 60)

# --------------------------------------------
# 첫 번째 코드: Feature Engineering
# --------------------------------------------
def make_basic_features_lstm(df):
    """시간 정보만 추가 (누수 없는 feature)"""
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))
    return df

def make_features_lstm(df):
    """전체 feature 생성"""
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))

    # 0인 날(휴무일) 간단한 binary feature만 추가
    df["IsZeroSales"] = (df["일매출"] == 0).astype(int)

    for lag in [1,2,3,7,14,28]:
        df[f"Lag{lag}"] = df["일매출"].shift(lag)
    for w in [7,14,28]:
        df[f"Mean{w}"] = df["일매출"].rolling(w).mean()
        df[f"Std{w}"] = df["일매출"].rolling(w).std()
    return df.fillna(0)

train_lstm = make_features_lstm(train.copy())

# Train/Validation Split
train_df_lstm, val_df_lstm = train_test_split(train_lstm, test_size=0.2, shuffle=False, random_state=SEED)

# Feature Selection
numeric_cols_lstm = train_df_lstm.select_dtypes(include=[np.number]).columns.tolist()
string_cols_lstm = train_df_lstm.select_dtypes(include=['object']).columns.tolist()

meta_cols_lstm = [
    c for c in numeric_cols_lstm
    if c not in ["일매출", "sales_scaled"]
    and not c.startswith("Lag")
    and not c.startswith("Mean")
    and not c.startswith("Std")
]
meta_cols_lstm = [c for c in meta_cols_lstm if c not in string_cols_lstm]

tree_features_lstm = meta_cols_lstm + [c for c in train_df_lstm.columns
                                      if (c.startswith("Lag") or c.startswith("Mean") or c.startswith("Std"))]
tree_features_lstm = [c for c in tree_features_lstm if c not in string_cols_lstm]

# Scaling
sales_scaler_lstm = MinMaxScaler()
meta_scaler_lstm = MinMaxScaler()

train_df_lstm["sales_scaled"] = sales_scaler_lstm.fit_transform(train_df_lstm[["일매출"]])
train_df_lstm[meta_cols_lstm] = meta_scaler_lstm.fit_transform(train_df_lstm[meta_cols_lstm])

val_df_lstm["sales_scaled"] = sales_scaler_lstm.transform(val_df_lstm[["일매출"]])
val_df_lstm[meta_cols_lstm] = meta_scaler_lstm.transform(val_df_lstm[meta_cols_lstm])

# Sequence Dataset
LOOKBACK_LSTM = 21
BATCH_LSTM = 16
EPOCHS_LSTM = 100

def create_seq_lstm(df, lookback):
    X, y = [], []
    sales = df["sales_scaled"].values
    meta = df[meta_cols_lstm].values
    for i in range(len(df)-lookback):
        seq = np.concatenate([
            sales[i:i+lookback].reshape(lookback,1),
            meta[i:i+lookback]
        ], axis=1)
        X.append(seq)
        y.append(sales[i+lookback])
    return np.array(X), np.array(y)

X_train_lstm, y_train_lstm = create_seq_lstm(train_df_lstm, LOOKBACK_LSTM)
X_val_lstm, y_val_lstm = create_seq_lstm(val_df_lstm, LOOKBACK_LSTM)

X_train_lstm = torch.tensor(X_train_lstm, dtype=torch.float32).to(DEVICE)
y_train_lstm = torch.tensor(y_train_lstm, dtype=torch.float32).to(DEVICE)
X_val_lstm = torch.tensor(X_val_lstm, dtype=torch.float32).to(DEVICE)
y_val_lstm = torch.tensor(y_val_lstm, dtype=torch.float32).to(DEVICE)

# Models
input_dim_lstm = 1 + len(meta_cols_lstm)

class MetaLSTM(nn.Module):
    def __init__(self, input_dim, hidden=128, num_layers=2, dropout=0.4):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc1 = nn.Linear(hidden, hidden // 2)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden // 2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1]
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out)

lstm_model = MetaLSTM(input_dim_lstm, hidden=128, num_layers=2, dropout=0.4).to(DEVICE)
opt_lstm = torch.optim.AdamW(lstm_model.parameters(), lr=0.0005, weight_decay=0)
scheduler_lstm = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_lstm, mode='min', factor=0.5, patience=5)
criterion = nn.MSELoss()

# DataLoader에 시드 고정
def seed_worker(worker_id):
    worker_seed = SEED % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

generator_lstm = torch.Generator()
generator_lstm.manual_seed(SEED)

train_dataset_lstm = TensorDataset(X_train_lstm, y_train_lstm)
train_loader_lstm = DataLoader(train_dataset_lstm, batch_size=BATCH_LSTM, shuffle=True, generator=generator_lstm, worker_init_fn=seed_worker)

# Train
early_stop_lstm = EarlyStopping(patience=15, min_delta=0.0001)
print("=" * 50)
print("LSTM 학습 시작")
print("=" * 50)

for ep in range(EPOCHS_LSTM):
    lstm_model.train()
    train_loss_lstm = 0

    for batch_x, batch_y in train_loader_lstm:
        opt_lstm.zero_grad()
        pred_l = lstm_model(batch_x).squeeze()
        loss_l = criterion(pred_l, batch_y)
        loss_l.backward()
        torch.nn.utils.clip_grad_norm_(lstm_model.parameters(), max_norm=1.0)
        opt_lstm.step()
        train_loss_lstm += loss_l.item()

    lstm_model.eval()
    with torch.no_grad():
        val_pred_l = lstm_model(X_val_lstm).squeeze()
        val_loss_lstm = criterion(val_pred_l, y_val_lstm).item()

    scheduler_lstm.step(val_loss_lstm)

    if (ep + 1) % 10 == 0 or ep == 0:
        print(f"[Epoch {ep+1}/{EPOCHS_LSTM}] LSTM - Val: {val_loss_lstm:.6f}")

    if early_stop_lstm(val_loss_lstm, lstm_model):
        print(f"\n✅ LSTM Early Stopping at epoch {ep+1}!")
        early_stop_lstm.load_best_model(lstm_model)
        break

early_stop_lstm.load_best_model(lstm_model)
print("✅ LSTM 학습 완료!")

# PART B: LSTM 예측
train_full_lstm = make_features_lstm(train.copy())
train_full_scaled_lstm = train_full_lstm.copy()
train_full_scaled_lstm["sales_scaled"] = sales_scaler_lstm.transform(train_full_lstm[["일매출"]])
train_full_scaled_lstm[meta_cols_lstm] = meta_scaler_lstm.transform(train_full_lstm[meta_cols_lstm])

future_dates = pd.date_range(start=test["영업일자"].min(), end=test["영업일자"].max())
future_df_lstm = pd.DataFrame({"영업일자": future_dates})
future_df_lstm = future_df_lstm.merge(acad, on="영업일자", how="left")

for col in meta_cols_lstm:
    if col not in future_df_lstm.columns:
        future_df_lstm[col] = 0

future_meta_lstm = meta_scaler_lstm.transform(future_df_lstm[meta_cols_lstm])
future_meta_lstm = pd.DataFrame(future_meta_lstm, columns=meta_cols_lstm)

def predict_nn_autoreg_lstm(model, train_df, future_meta_df, lookback):
    """첫 번째 코드: LSTM용 autoregressive 예측"""
    model.eval()
    history_sales = list(train_df["sales_scaled"].values)
    history_meta = train_df[meta_cols_lstm].values.tolist()
    preds_raw = []

    with torch.no_grad():
        for i in range(len(future_meta_df)):
            seq_sales = np.array(history_sales[-lookback:]).reshape(lookback, 1)
            seq_meta = np.array(history_meta[-lookback:])
            seq = np.concatenate([seq_sales, seq_meta], axis=1)

            X = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(DEVICE)
            scaled_pred = model(X).item()
            raw_pred = sales_scaler_lstm.inverse_transform([[scaled_pred]])[0, 0]

            preds_raw.append(raw_pred)

            next_scaled = sales_scaler_lstm.transform([[raw_pred]])[0, 0]
            history_sales.append(next_scaled)
            history_meta.append(future_meta_df.iloc[i].values.tolist())

    return np.array(preds_raw)

future_lstm_raw = predict_nn_autoreg_lstm(lstm_model, train_full_scaled_lstm, future_meta_lstm, LOOKBACK_LSTM)

# 첫 번째 코드: 후처리
def postprocess_zero_days_lstm(preds, future_dates, train_df, threshold_ratio=0.7, small_pred_threshold=10000):
    """첫 번째 코드: 0인 날 패턴 학습 및 적용"""
    preds = preds.copy()
    train_copy = train_df.copy()

    train_copy["MonthDay"] = train_copy["영업일자"].dt.strftime("%m-%d")
    train_copy["IsZero"] = (train_copy["일매출"] == 0).astype(int)
    zero_ratio_by_date = train_copy.groupby("MonthDay")["IsZero"].mean()

    for i, date in enumerate(future_dates):
        month_day = date.strftime("%m-%d")
        if month_day in zero_ratio_by_date.index:
            zero_ratio = zero_ratio_by_date[month_day]
            if zero_ratio >= threshold_ratio and preds[i] < small_pred_threshold:
                preds[i] = 0
    return preds

future_lstm_post = postprocess_zero_days_lstm(future_lstm_raw, future_dates, train_full_lstm)
future_lstm = future_lstm_post
print(f"LSTM 예측 완료 - 범위: {future_lstm.min():.2f} ~ {future_lstm.max():.2f}")

# ============================================================================
# ============================================================================
# PART 2: 두 번째 코드 - GRU (데이터 누수 수정)
# ============================================================================
# ============================================================================

print("\n" + "=" * 60)
print("PART 2: GRU 모델 학습 및 예측 (데이터 누수 수정)")
print("=" * 60)

# 두 번째 코드의 Feature Engineering
def make_features_gru(df):
    """두 번째 코드: 전체 feature 생성"""
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))
    df["IsZeroSales"] = (df["일매출"] == 0).astype(int)

    for lag in [1,2,3,7,14,28]:
        df[f"Lag{lag}"] = df["일매출"].shift(lag)
    for w in [7,14,28]:
        df[f"Mean{w}"] = df["일매출"].rolling(w).mean()
        df[f"Std{w}"] = df["일매출"].rolling(w).std()

    df["Mean3"] = df["일매출"].rolling(3).mean()
    df["Max7"] = df["일매출"].rolling(7).max()
    df["Min7"] = df["일매출"].rolling(7).min()
    df["CV7"] = df["Std7"] / (df["Mean7"] + 1e-8)

    if len(df) > 0:
        month_avg = df.groupby("Month")["일매출"].mean()
        df["MonthAvg"] = df["Month"].map(month_avg)
        weekday_avg = df.groupby("DayOfWeek")["일매출"].mean()
        df["WeekdayAvg"] = df["DayOfWeek"].map(weekday_avg)

    return df.fillna(0)

# ✅ 수정: train의 평균값을 미리 계산 (데이터 누수 방지)
def make_features_gru_safe(df, month_avg_train=None, weekday_avg_train=None):
    """데이터 누수 없는 GRU feature 생성 (train의 평균값 사용)"""
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))
    df["IsZeroSales"] = (df["일매출"] == 0).astype(int)

    for lag in [1,2,3,7,14,28]:
        df[f"Lag{lag}"] = df["일매출"].shift(lag)
    for w in [7,14,28]:
        df[f"Mean{w}"] = df["일매출"].rolling(w).mean()
        df[f"Std{w}"] = df["일매출"].rolling(w).std()

    df["Mean3"] = df["일매출"].rolling(3).mean()
    df["Max7"] = df["일매출"].rolling(7).max()
    df["Min7"] = df["일매출"].rolling(7).min()
    df["CV7"] = df["Std7"] / (df["Mean7"] + 1e-8)

    # ✅ train의 평균값 사용 (test 데이터 누수 방지)
    if month_avg_train is not None:
        df["MonthAvg"] = df["Month"].map(month_avg_train).fillna(0)
    else:
        if len(df) > 0:
            month_avg = df.groupby("Month")["일매출"].mean()
            df["MonthAvg"] = df["Month"].map(month_avg).fillna(0)

    if weekday_avg_train is not None:
        df["WeekdayAvg"] = df["DayOfWeek"].map(weekday_avg_train).fillna(0)
    else:
        if len(df) > 0:
            weekday_avg = df.groupby("DayOfWeek")["일매출"].mean()
            df["WeekdayAvg"] = df["DayOfWeek"].map(weekday_avg).fillna(0)

    return df.fillna(0)

train_gru = make_features_gru(train.copy())
train_df_gru, val_df_gru = train_test_split(train_gru, test_size=0.2, shuffle=False, random_state=SEED)

# Feature Selection
numeric_cols_gru = train_df_gru.select_dtypes(include=[np.number]).columns.tolist()
string_cols_gru = train_df_gru.select_dtypes(include=['object']).columns.tolist()

meta_cols_gru = [
    c for c in numeric_cols_gru
    if c not in ["일매출", "sales_scaled"]
    and not c.startswith("Lag")
    and not c.startswith("Mean")
    and not c.startswith("Std")
    and not c.startswith("Max")
    and not c.startswith("Min")
    and not c.startswith("CV")
    and not c.startswith("MonthAvg")
    and not c.startswith("WeekdayAvg")
    and c not in acad_cols
]
meta_cols_gru = [c for c in meta_cols_gru if c not in string_cols_gru]

tree_features_gru = meta_cols_gru + [c for c in train_df_gru.columns
                                     if (c.startswith("Lag") or c.startswith("Mean") or
                                         c.startswith("Std") or c.startswith("Max") or
                                         c.startswith("Min") or c.startswith("CV") or
                                         c.startswith("MonthAvg") or c.startswith("WeekdayAvg"))]
tree_features_gru = [c for c in tree_features_gru if c not in string_cols_gru]

# Scaling
sales_scaler_gru = MinMaxScaler()
meta_scaler_gru = MinMaxScaler()

train_df_gru["sales_scaled"] = sales_scaler_gru.fit_transform(train_df_gru[["일매출"]])
train_df_gru[meta_cols_gru] = meta_scaler_gru.fit_transform(train_df_gru[meta_cols_gru])

val_df_gru["sales_scaled"] = sales_scaler_gru.transform(val_df_gru[["일매출"]])
val_df_gru[meta_cols_gru] = meta_scaler_gru.transform(val_df_gru[meta_cols_gru])

# Sequence Dataset
LOOKBACK_GRU = 7
BATCH_GRU = 64
EPOCHS_GRU = 200

def create_seq_gru(df, lookback):
    X, y = [], []
    sales = df["sales_scaled"].values
    meta = df[meta_cols_gru].values
    for i in range(len(df)-lookback):
        seq = np.concatenate([
            sales[i:i+lookback].reshape(lookback,1),
            meta[i:i+lookback]
        ], axis=1)
        X.append(seq)
        y.append(sales[i+lookback])
    return np.array(X), np.array(y)

X_train_gru, y_train_gru = create_seq_gru(train_df_gru, LOOKBACK_GRU)
X_val_gru, y_val_gru = create_seq_gru(val_df_gru, LOOKBACK_GRU)

X_train_gru = torch.tensor(X_train_gru, dtype=torch.float32).to(DEVICE)
y_train_gru = torch.tensor(y_train_gru, dtype=torch.float32).to(DEVICE)
X_val_gru = torch.tensor(X_val_gru, dtype=torch.float32).to(DEVICE)
y_val_gru = torch.tensor(y_val_gru, dtype=torch.float32).to(DEVICE)

# Models
input_dim_gru = 1 + len(meta_cols_gru)

class MetaGRU(nn.Module):
    def __init__(self, input_dim, hidden=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc1 = nn.Linear(hidden, hidden // 2)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden // 2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1]
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out)

gru_model = MetaGRU(input_dim_gru, hidden=64, num_layers=2, dropout=0.3).to(DEVICE)
opt_gru = torch.optim.AdamW(gru_model.parameters(), lr=0.0001, weight_decay=0)
scheduler_gru = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_gru, mode='min', factor=0.5, patience=5)

generator_gru = torch.Generator()
generator_gru.manual_seed(SEED)

train_dataset_gru = TensorDataset(X_train_gru, y_train_gru)
train_loader_gru = DataLoader(train_dataset_gru, batch_size=BATCH_GRU, shuffle=True, generator=generator_gru, worker_init_fn=seed_worker)

# Train
early_stop_gru = EarlyStopping(patience=15, min_delta=0.0001)
print("=" * 50)
print("GRU 학습 시작")
print("=" * 50)

for ep in range(EPOCHS_GRU):
    gru_model.train()
    train_loss_gru = 0

    for batch_x, batch_y in train_loader_gru:
        opt_gru.zero_grad()
        pred_g = gru_model(batch_x).squeeze()
        loss_g = criterion(pred_g, batch_y)
        loss_g.backward()
        torch.nn.utils.clip_grad_norm_(gru_model.parameters(), max_norm=0.5)
        opt_gru.step()
        train_loss_gru += loss_g.item()

    gru_model.eval()
    with torch.no_grad():
        val_pred_g = gru_model(X_val_gru).squeeze()
        val_loss_gru = criterion(val_pred_g, y_val_gru).item()

    scheduler_gru.step(val_loss_gru)

    if (ep + 1) % 20 == 0:
        print(f"Epoch {ep+1}/{EPOCHS_GRU} - GRU: {val_loss_gru:.6f}")

    if early_stop_gru(val_loss_gru, gru_model):
        print(f"GRU Early stopping at epoch {ep+1}")
        early_stop_gru.load_best_model(gru_model)
        break

early_stop_gru.load_best_model(gru_model)
print("✅ GRU 학습 완료!")

# PART B: GRU Direct Multi-step 예측 (데이터 누수 수정)
train_full_gru = make_features_gru(train.copy())

# ✅ train의 평균값 미리 계산 (데이터 누수 방지)
train_full_gru_for_avg = make_features_gru(train.copy())
month_avg_train_gru = train_full_gru_for_avg.groupby("Month")["일매출"].mean()
weekday_avg_train_gru = train_full_gru_for_avg.groupby("DayOfWeek")["일매출"].mean()

train_full_scaled_gru = train_full_gru.copy()
train_full_scaled_gru["sales_scaled"] = sales_scaler_gru.transform(train_full_gru[["일매출"]])
train_full_scaled_gru[meta_cols_gru] = meta_scaler_gru.transform(train_full_gru[meta_cols_gru])

# ✅ 수정: test 데이터 사용 안 함, 미래 날짜만 생성
future_dates_gru = pd.date_range(start=test["영업일자"].min(), end=test["영업일자"].max())
future_df_gru = pd.DataFrame({"영업일자": future_dates_gru})
future_df_gru = future_df_gru.merge(acad, on="영업일자", how="left")
# 학사일정 컬럼 추가
for col in acad_cols:
    if col not in future_df_gru.columns:
        future_df_gru[col] = 0

# meta_cols_gru에 필요한 기본 컬럼 확인
for col in meta_cols_gru:
    if col not in future_df_gru.columns:
        future_df_gru[col] = 0

def predict_gru_direct_safe(model, train_df, future_df, lookback, month_avg_train, weekday_avg_train):
    """✅ 데이터 누수 없는 GRU Direct Multi-step 예측"""
    model.eval()
    preds = []

    # history 초기화 (train 데이터만 사용)
    history_df = train_df.copy()

    for i in range(len(future_df)):
        row = future_df.iloc[i]

        # 필요한 경우에만 feature 생성 (train 평균값 사용)
        if i < lookback:
            hist_for_gru = history_df.tail(lookback - i - 1).copy()
            current_row = pd.DataFrame([{
                "영업일자": row["영업일자"],
                "일매출": history_df["일매출"].tail(7).mean() if len(history_df) >= 7 else history_df["일매출"].mean(),
                **{col: row[col] if col in row.index else 0 for col in meta_cols_gru}
            }])
            seq_df_gru = pd.concat([hist_for_gru, current_row], ignore_index=True)
        else:
            hist_for_gru = history_df.tail(1).copy()
            temp_rows = []
            for j in range(1, lookback):
                if i - j >= 0:
                    temp_row = pd.DataFrame([{
                        "영업일자": future_df.iloc[i-j]["영업일자"],
                        "일매출": preds[i-j] if i-j < len(preds) else history_df["일매출"].tail(7).mean(),
                        **{col: future_df.iloc[i-j][col] if col in future_df.columns else 0 for col in meta_cols_gru}
                    }])
                    temp_rows.append(temp_row)

            if temp_rows:
                seq_df_gru = pd.concat([hist_for_gru] + temp_rows, ignore_index=True)
            else:
                seq_df_gru = hist_for_gru

        # 길이 보장
        if len(seq_df_gru) < lookback:
            last_row = seq_df_gru.iloc[-1:].copy()
            padding_count = lookback - len(seq_df_gru)
            padding = pd.concat([last_row] * padding_count, ignore_index=True)
            seq_df_gru = pd.concat([seq_df_gru, padding], ignore_index=True)
        elif len(seq_df_gru) > lookback:
            seq_df_gru = seq_df_gru.tail(lookback).reset_index(drop=True)

        # ✅ train의 평균값을 사용하여 feature 생성 (데이터 누수 방지)
        seq_df_gru = make_features_gru_safe(seq_df_gru, month_avg_train, weekday_avg_train)
        seq_df_gru["sales_scaled"] = sales_scaler_gru.transform(seq_df_gru[["일매출"]].fillna(0))
        seq_df_gru[meta_cols_gru] = meta_scaler_gru.transform(seq_df_gru[meta_cols_gru].fillna(0))

        # Sequence 생성
        sales_seq_gru = seq_df_gru["sales_scaled"].values.reshape(lookback, 1)
        meta_seq_gru = seq_df_gru[meta_cols_gru].values
        seq_gru = np.concatenate([sales_seq_gru, meta_seq_gru], axis=1).reshape(1, lookback, -1)

        # 예측
        with torch.no_grad():
            X_gru = torch.tensor(seq_gru, dtype=torch.float32).to(DEVICE)
            pred_scaled_gru = model(X_gru).squeeze().cpu().item()
            pred_gru = sales_scaler_gru.inverse_transform([[pred_scaled_gru]])[0][0]
            pred_gru = max(0, pred_gru)

        preds.append(pred_gru)

        # history 업데이트 (예측값 사용)
        new_row = pd.DataFrame([{
            "영업일자": row["영업일자"],
            "일매출": pred_gru,
            **{col: row[col] if col in row.index else 0 for col in meta_cols_gru}
        }])
        history_df = pd.concat([history_df, new_row], ignore_index=True)

    return np.array(preds)

future_gru = predict_gru_direct_safe(
    gru_model,
    train_full_scaled_gru,
    future_df_gru,
    LOOKBACK_GRU,
    month_avg_train_gru,
    weekday_avg_train_gru
)
print(f"GRU 예측 완료 - 범위: {future_gru.min():.2f} ~ {future_gru.max():.2f}")

# ============================================================================
# ============================================================================
# PART 3: 세 번째 코드 - Tree 모델들 (그대로 이어붙이기)
# ============================================================================
# ============================================================================

print("\n" + "=" * 60)
print("PART 3: Tree 모델 학습 및 예측 (세 번째 코드 그대로)")
print("=" * 60)

# 세 번째 코드의 Feature Engineering
def make_features_tree(df):
    """세 번째 코드: 전체 feature 생성"""
    df = df.copy()
    df["DayOfWeek"] = df["영업일자"].dt.dayofweek
    df["Month"] = df["영업일자"].dt.month
    df["Day"] = df["영업일자"].dt.day
    df["IsWeekend"] = (df["DayOfWeek"] >= 5).astype(int)
    df["OpHoursFactor"] = df["DayOfWeek"].apply(lambda x: 1 if x<5 else (0.55 if x==5 else 0))
    df["IsZeroSales"] = (df["일매출"] == 0).astype(int)

    for lag in [1,2,3,7,14,28]:
        df[f"Lag{lag}"] = df["일매출"].shift(lag)
    for w in [7,14,28]:
        df[f"Mean{w}"] = df["일매출"].rolling(w).mean()
        df[f"Std{w}"] = df["일매출"].rolling(w).std()

    df["Mean3"] = df["일매출"].rolling(3).mean()
    df["Max7"] = df["일매출"].rolling(7).max()
    df["Min7"] = df["일매출"].rolling(7).min()
    df["CV7"] = df["Std7"] / (df["Mean7"] + 1e-8)

    if len(df) > 0:
        month_avg = df.groupby("Month")["일매출"].mean()
        df["MonthAvg"] = df["Month"].map(month_avg)
        weekday_avg = df.groupby("DayOfWeek")["일매출"].mean()
        df["WeekdayAvg"] = df["DayOfWeek"].map(weekday_avg)

    return df.fillna(0)

train_tree = make_features_tree(train.copy())
train_df_tree, val_df_tree = train_test_split(train_tree, test_size=0.2, shuffle=False, random_state=SEED)

# Feature Selection
numeric_cols_tree = train_df_tree.select_dtypes(include=[np.number]).columns.tolist()
string_cols_tree = train_df_tree.select_dtypes(include=['object']).columns.tolist()

meta_cols_tree = [
    c for c in numeric_cols_tree
    if c not in ["일매출", "sales_scaled"]
    and not c.startswith("Lag")
    and not c.startswith("Mean")
    and not c.startswith("Std")
    and not c.startswith("Max")
    and not c.startswith("Min")
    and not c.startswith("CV")
    and not c.startswith("MonthAvg")
    and not c.startswith("WeekdayAvg")
    and c not in acad_cols
]
meta_cols_tree = [c for c in meta_cols_tree if c not in string_cols_tree]

tree_features_tree = meta_cols_tree + [c for c in train_df_tree.columns
                                       if (c.startswith("Lag") or c.startswith("Mean") or
                                           c.startswith("Std") or c.startswith("Max") or
                                           c.startswith("Min") or c.startswith("CV") or
                                           c.startswith("MonthAvg") or c.startswith("WeekdayAvg"))]
tree_features_tree = [c for c in tree_features_tree if c not in string_cols_tree]

# Tree 모델 학습
lgb_params_tree = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 2.0,
    'max_depth': 10,
    'early_stopping_rounds': 50,
    'verbose': -1,
    'force_col_wise': True,
    'random_state': SEED,
    'deterministic': True
}

xgb_params_tree = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'learning_rate': 0.01,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 5,
    'gamma': 0.1,
    'reg_alpha': 0.1,
    'reg_lambda': 2.0,
    'tree_method': 'hist',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'random_state': SEED,
    'deterministic': True
}

lgb_train_tree = lgb.Dataset(train_df_tree[tree_features_tree], label=train_df_tree["일매출"])
lgb_val_tree = lgb.Dataset(val_df_tree[tree_features_tree], label=val_df_tree["일매출"], reference=lgb_train_tree)

print("LightGBM 학습 중...")
lgb_model_tree = lgb.train(
    lgb_params_tree,
    lgb_train_tree,
    valid_sets=[lgb_train_tree, lgb_val_tree],
    valid_names=['train', 'val'],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

xgb_train_tree = xgb.DMatrix(train_df_tree[tree_features_tree], label=train_df_tree["일매출"])
xgb_val_tree = xgb.DMatrix(val_df_tree[tree_features_tree], label=val_df_tree["일매출"])

print("\nXGBoost 학습 중...")
xgb_model_tree = xgb.train(
    xgb_params_tree,
    xgb_train_tree,
    num_boost_round=1000,
    evals=[(xgb_train_tree, 'train'), (xgb_val_tree, 'val')],
    early_stopping_rounds=50,
    verbose_eval=100
)

# 전체 train으로 재학습
train_full_tree = make_features_tree(train.copy())

for col in tree_features_tree:
    if col not in train_full_tree.columns:
        train_full_tree[col] = 0

lgb_best_iter_tree = lgb_model_tree.best_iteration if hasattr(lgb_model_tree, 'best_iteration') else 200
xgb_best_iter_tree = xgb_model_tree.best_iteration if hasattr(xgb_model_tree, 'best_iteration') and xgb_model_tree.best_iteration is not None else 200

print(f"\nLightGBM 전체 데이터로 재학습 중... (iteration: {lgb_best_iter_tree})")
lgb_params_full_tree = lgb_params_tree.copy()
lgb_params_full_tree.pop('early_stopping_rounds', None)
lgb_train_full_tree = lgb.Dataset(train_full_tree[tree_features_tree], label=train_full_tree["일매출"])
lgb_model_full_tree = lgb.train(lgb_params_full_tree, lgb_train_full_tree, num_boost_round=lgb_best_iter_tree)

print(f"\nXGBoost 전체 데이터로 재학습 중... (iteration: {xgb_best_iter_tree})")
xgb_train_full_tree = xgb.DMatrix(train_full_tree[tree_features_tree], label=train_full_tree["일매출"])
xgb_model_full_tree = xgb.train(xgb_params_tree, xgb_train_full_tree, num_boost_round=xgb_best_iter_tree, verbose_eval=0)

# Tree autoregressive 예측
def predict_tree_autoreg(model, model_type, history_df, n_steps, xgb_best_iter=None):
    """세 번째 코드: Tree 모델 자동회귀 예측"""
    preds = []
    temp_df = history_df.copy().reset_index(drop=True)

    for i in range(n_steps):
        if len(temp_df) == 0:
            preds.append(0)
            continue

        for feat in tree_features_tree:
            if feat not in temp_df.columns:
                temp_df[feat] = 0

        if len(temp_df) >= 7:
            recent_7d_avg = temp_df["일매출"].tail(7).mean()
        else:
            recent_7d_avg = temp_df["일매출"].mean() if len(temp_df) > 0 else 0

        lag1 = temp_df["일매출"].iloc[-1] if len(temp_df) > 0 else 0

        try:
            if model_type == 'lgb':
                last_row = temp_df[tree_features_tree].iloc[-1:].copy()
                pred = model.predict(last_row, num_iteration=model.best_iteration if hasattr(model, 'best_iteration') else None)[0]
            else:
                last_row = temp_df[tree_features_tree].iloc[-1:].copy()
                if xgb_best_iter is not None and xgb_best_iter > 0:
                    pred = model.predict(xgb.DMatrix(last_row), iteration_range=(0, xgb_best_iter))[0]
                else:
                    pred = model.predict(xgb.DMatrix(last_row))[0]
        except Exception as e:
            pred = lag1 if lag1 > 0 else recent_7d_avg

        pred = max(0, pred)
        raw_pred = pred

        # 스케일 보정
        if pred < recent_7d_avg * 0.15 and recent_7d_avg > 0:
            if lag1 > 0:
                pred = max(lag1 * 0.4, min(lag1 * 1.3, pred * (lag1 / max(pred, 1))))
            else:
                pred = max(recent_7d_avg * 0.3, min(recent_7d_avg * 1.1, pred * (recent_7d_avg / max(pred, 1))))

        if pred < 10000 and recent_7d_avg > 50000:
            pred = max(10000, pred)

        preds.append(pred)

        if len(temp_df) > 0:
            new_row = temp_df.iloc[-1].copy()
            new_row["일매출"] = pred
            if '영업일자' in new_row and pd.notna(new_row['영업일자']):
                new_row['영업일자'] = new_row['영업일자'] + pd.Timedelta(days=1)
            temp_df = pd.concat([temp_df, pd.DataFrame([new_row])], ignore_index=True)
            temp_df = make_features_tree(temp_df)
            temp_df = temp_df.reset_index(drop=True)

    return np.array(preds)

print("\nTree 모델 예측 중...")
future_lgb = predict_tree_autoreg(lgb_model_full_tree, 'lgb', train_full_tree, len(test))
future_xgb = predict_tree_autoreg(xgb_model_full_tree, 'xgb', train_full_tree, len(test), xgb_best_iter_tree)
future_tree = (future_lgb + future_xgb) / 2
print(f"Tree 예측 완료 - LGB 범위: {future_lgb.min():.2f} ~ {future_lgb.max():.2f}")
print(f"Tree 예측 완료 - XGB 범위: {future_xgb.min():.2f} ~ {future_xgb.max():.2f}")

# ============================================================================
# ============================================================================
# 최종 앙상블 (Scale 맞춤)
# ============================================================================
# ============================================================================

print("\n" + "=" * 60)
print("최종 앙상블 (Scale 조정)")
print("=" * 60)

# 각 모델 예측값 스케일 확인
print(f"\n각 모델 예측값 통계:")
print(f"  LSTM - 평균: {future_lstm.mean():.2f}, 최소: {future_lstm.min():.2f}, 최대: {future_lstm.max():.2f}")
print(f"  GRU  - 평균: {future_gru.mean():.2f}, 최소: {future_gru.min():.2f}, 최대: {future_gru.max():.2f}")
print(f"  Tree - 평균: {future_tree.mean():.2f}, 최소: {future_tree.min():.2f}, 최대: {future_tree.max():.2f}")

# NN 앙상블
future_nn_ensemble = (future_lstm + future_gru) / 2

# 최종 앙상블 (0.3*Tree + 0.7*NN)
future_final = 0.3 * future_tree + 0.7 * future_nn_ensemble

print(f"\n최종 앙상블 - 평균: {future_final.mean():.2f}, 최소: {future_final.min():.2f}, 최대: {future_final.max():.2f}")

# ============================================
# TEST 파일 로드 및 성능 평가
# ============================================
matched = test[test["일매출"].notna()].copy()
matched["일매출"] = pd.to_numeric(matched["일매출"], errors='coerce').fillna(0).astype(float)

pred_df = pd.DataFrame({
    "영업일자": future_dates[:len(matched)],
    "예측_LSTM": future_lstm[:len(matched)],
    "예측_GRU": future_gru[:len(matched)],
    "예측_NN_Ensemble": future_nn_ensemble[:len(matched)],
    "예측_LGB": future_lgb[:len(matched)],
    "예측_XGB": future_xgb[:len(matched)],
    "Tree_Ensemble": future_tree[:len(matched)],
    "Final_Ensemble": future_final[:len(matched)]
})

result_df = matched.merge(pred_df, on="영업일자", how="inner")

if len(result_df) == 0:
    print("⚠️ 경고: 예측 날짜와 test 날짜가 일치하지 않습니다!")
else:
    print(f"\n✅ 날짜 매칭 완료! (매칭된 개수: {len(result_df)})")

# 성능 평가 함수
def MAE(a, b): return np.mean(np.abs(a-b))
def RMSE(a, b): return np.sqrt(np.mean((a-b)**2))
def SMAPE(a, b): return np.mean(2 * np.abs(a-b) / (np.abs(a)+np.abs(b)+1e-9)) * 100

if len(result_df) > 0:
    y_true = result_df["일매출"].values
    print("\n" + "="*20 + " TEST 성능 " + "="*20)

    print("\n--- Neural Networks ---")
    print(f"LSTM MAE   : {MAE(y_true, result_df['예측_LSTM'].values):.2f}")
    print(f"LSTM RMSE  : {RMSE(y_true, result_df['예측_LSTM'].values):.2f}")
    print(f"LSTM SMAPE : {SMAPE(y_true, result_df['예측_LSTM'].values):.2f}%\n")

    print(f"GRU  MAE   : {MAE(y_true, result_df['예측_GRU'].values):.2f}")
    print(f"GRU  RMSE  : {RMSE(y_true, result_df['예측_GRU'].values):.2f}")
    print(f"GRU  SMAPE : {SMAPE(y_true, result_df['예측_GRU'].values):.2f}%\n")

    print("--- NN Ensemble (LSTM + GRU) ---")
    print(f"NN_Ensemble MAE   : {MAE(y_true, result_df['예측_NN_Ensemble'].values):.2f}")
    print(f"NN_Ensemble RMSE  : {RMSE(y_true, result_df['예측_NN_Ensemble'].values):.2f}")
    print(f"NN_Ensemble SMAPE : {SMAPE(y_true, result_df['예측_NN_Ensemble'].values):.2f}%\n")

    print("\n--- Tree Models ---")
    print(f"LGB  MAE   : {MAE(y_true, result_df['예측_LGB'].values):.2f}")
    print(f"LGB  RMSE  : {RMSE(y_true, result_df['예측_LGB'].values):.2f}")
    print(f"LGB  SMAPE : {SMAPE(y_true, result_df['예측_LGB'].values):.2f}%\n")

    print(f"XGB  MAE   : {MAE(y_true, result_df['예측_XGB'].values):.2f}")
    print(f"XGB  RMSE  : {RMSE(y_true, result_df['예측_XGB'].values):.2f}")
    print(f"XGB  SMAPE : {SMAPE(y_true, result_df['예측_XGB'].values):.2f}%\n")

    print("\n--- Tree Ensemble ---")
    print(f"Tree MAE   : {MAE(y_true, result_df['Tree_Ensemble'].values):.2f}")
    print(f"Tree RMSE  : {RMSE(y_true, result_df['Tree_Ensemble'].values):.2f}")
    print(f"Tree SMAPE : {SMAPE(y_true, result_df['Tree_Ensemble'].values):.2f}%\n")

    print("\n--- Final Ensemble (0.3*Tree + 0.7*NN_Ensemble) ---")
    print(f"MAE   : {MAE(y_true, result_df['Final_Ensemble'].values):.2f}")
    print(f"RMSE  : {RMSE(y_true, result_df['Final_Ensemble'].values):.2f}")
    print(f"SMAPE : {SMAPE(y_true, result_df['Final_Ensemble'].values):.2f}%")
    print("="*52)

# CSV 저장
output = pd.DataFrame({
    "영업일자": result_df["영업일자"].values if len(result_df) > 0 else test["영업일자"].values,
    "일매출": result_df["일매출"].values if len(result_df) > 0 else matched["일매출"].values,
    "예측_LSTM": future_lstm[:len(test)],
    "예측_GRU": future_gru[:len(test)],
    "예측_NN_Ensemble": future_nn_ensemble[:len(test)],
    "예측_LGB": future_lgb[:len(test)],
    "예측_XGB": future_xgb[:len(test)],
    "Tree_Ensemble": future_tree[:len(test)],
    "Final_Ensemble": future_final[:len(test)]
})

output.to_csv("final_test_prediction_optimal.csv", index=False, encoding='utf-8-sig')
print("\n📌 저장 완료 → final_test_prediction_optimal.csv")
print(output.head(10))



데이터 로드 중...

학사일정 데이터 컬럼: ['weekday', 'weekend', 'holiday', 'semester', 'seasonal', 'exam', 'ceremony', 'dormitory']

PART 1: LSTM 모델 학습 및 예측 (첫 번째 코드 그대로)
LSTM 학습 시작
[Epoch 1/100] LSTM - Val: 0.024348
[Epoch 10/100] LSTM - Val: 0.009573
[Epoch 20/100] LSTM - Val: 0.016958
[Epoch 30/100] LSTM - Val: 0.006880
[Epoch 40/100] LSTM - Val: 0.010657
[Epoch 50/100] LSTM - Val: 0.007181
[Epoch 60/100] LSTM - Val: 0.006977

✅ LSTM Early Stopping at epoch 66!
✅ LSTM 학습 완료!
LSTM 예측 완료 - 범위: 5321.58 ~ 451870.89

PART 2: GRU 모델 학습 및 예측 (데이터 누수 수정)
GRU 학습 시작
Epoch 20/200 - GRU: 0.026271
Epoch 40/200 - GRU: 0.024101
Epoch 60/200 - GRU: 0.019360
Epoch 80/200 - GRU: 0.012692
Epoch 100/200 - GRU: 0.010577
Epoch 120/200 - GRU: 0.009785
Epoch 140/200 - GRU: 0.009750
GRU Early stopping at epoch 145
✅ GRU 학습 완료!
GRU 예측 완료 - 범위: 0.00 ~ 173461.38

PART 3: Tree 모델 학습 및 예측 (세 번째 코드 그대로)
LightGBM 학습 중...
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 80308.3	val's rmse: 65973.6
